In [ ]:
import pandas as pd
import bt
# List of file names
file_names = ["ACB.csv", "BID.csv"]

# List to store individual DataFrames
data = []

# Read each CSV file, set 'Date' column as index, and rename 'Close' column
for file_name in file_names:
    df = pd.read_csv(file_name)
    df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' column to DateTime format
    df.set_index('Date', inplace=True)  # Set 'Date' column as index
    df.rename(columns={'Close': file_name[:-4]}, inplace=True)  # Rename 'Close' column
    data.append(df)

# Merge all DataFrames on their indices
merged_df = pd.concat(data, axis=1)

# Select rows starting from 1-1-2018 and keep only the 'Close' column
data = merged_df.loc['2018-01-01':, ['ACB', 'BID']]  # Adjust column names if needed
data = data.dropna()

sma = data.rolling(50).mean()

plot = bt.merge(data, sma).plot(figsize=(15, 5))

class SelectWhere(bt.Algo):

    """
    Selects securities based on an indicator DataFrame.

    Selects securities where the value is True on the current date (target.now).

    Args:
        * signal (DataFrame): DataFrame containing the signal (boolean DataFrame)

    Sets:
        * selected

    """
    def __init__(self, signal):
        self.signal = signal

    def __call__(self, target):
        # get signal on target.now
        if target.now in self.signal.index:
            sig = self.signal.loc[target.now]

            # get indices where true as list
            selected = list(sig.index[sig])

            # save in temp - this will be used by the weighing algo
            target.temp['selected'] = selected

        # return True because we want to keep on moving down the stack
        return True
signal = data > sma
# first we create the Strategy
s = bt.Strategy('above50sma', [SelectWhere(data > sma),
                               bt.algos.WeighEqually(),
                               bt.algos.Rebalance()])

# now we create the Backtest
t = bt.Backtest(s, data)

# and let's run it!
res = bt.run(t)
res.plot();
res.display()